In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from modules.util.util import importstr, run_shell_cmd
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [ ]:
# data_path = 'COVID-19-20_v2/Train'     # replace with local data path here
data_path = '/home/fsuser/programming/datasets/COVID-19-20_v2/Train'

Generate metadata and cache files

In [ ]:
run_shell_cmd(log, 'prepcache.CovidPrepCacheApp', f'--data-path={data_path}', f'--num-workers={4}', f'--batch-size={20}')

Use lr finder to find a good learning rate

In [ ]:
run_shell_cmd(log, 'modules.util.lr_finder.LearningRateFinder')

Set the value of learning rate as found by the learning rate founder

In [ ]:
learning_rate = 1e-2

Begin training

In [ ]:
run_shell_cmd(log, 'training.CovidSegmentationTrainingApp', f'--lr={learning_rate}')